In [1]:
from xgboost import XGBClassifier, XGBRegressor
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

### Loading toy dataset and rename columns

In [2]:
data = load_iris()
columns = data.feature_names
columns = [c.replace(" ", "_").replace("(", "").replace(")", "") for c in columns]

In [3]:
X = data["data"]
y = data["target"]

### Apply min-max scaler to the dataset for correct work of quantization

In [4]:
X = MinMaxScaler().fit_transform(X)
X = pd.DataFrame(X, columns=columns)
X.head()

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm
0,0.222222,0.625000,0.067797,0.041667
1,0.166667,0.416667,0.067797,0.041667
2,0.111111,0.500000,0.050847,0.041667
3,0.083333,0.458333,0.084746,0.041667
4,0.194444,0.666667,0.067797,0.041667


### Splittting our data on train and test

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

### Training our model

In [6]:
model = XGBClassifier(n_estimators=6)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=6, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

### Score Xgboost classifier

In [7]:
from sklearn.metrics import f1_score

f1_score(y_test, model.predict(X_test), average="macro")

1.0

### Let's convert our model to Leo Smart-Contract

In [8]:
import sys
sys.path.append("../../leo-transpiler-python") # Provide path to leo-transpiler so, that it can be imported

In [9]:
from leo_transpiler.boostings import XgboostTranspiler

transpiler = XgboostTranspiler(model, quantize_bits=128)
transpiler

XgboostTranspiler()

### Let's single Xgboost tree transpilation in Leo Smart-Contract

### Now we can save the whole program to source directory

But first we need to initalize Aleo project

```shell
leo new iris
```

In [10]:
!leo new iris

       Leo ✅ Created an Aleo program 'iris' (in "/Users/danielto1404/Documents/crypto/aleo/aleo-recommender/examples/xgboost/iris")


In [11]:
transpiler.save_code("./iris/src", program_name="iris")

In [12]:
!cat iris/src/main.leo

program iris.aleo { 
     struct Probas { class_0_proba: i128, class_1_proba: i128, class_2_proba: i128 }

    function class_0_tree_0 ( sepal_length_cm: i128, sepal_width_cm: i128, petal_length_cm: i128, petal_width_cm: i128 ) -> i128 { 
        if ( petal_length_cm < 2266760870222161408i128 ) { 
            return 3912589863452533760i128; 
        } else { 
            return -2020486755692502528i128; 
        } 
    }

    function class_1_tree_0 ( sepal_length_cm: i128, sepal_width_cm: i128, petal_length_cm: i128, petal_width_cm: i128 ) -> i128 { 
        if ( petal_length_cm < 2266760870222161408i128 ) { 
            return -1956295203815741952i128; 
        } else { 
            if ( petal_width_cm < 6341068275337658368i128 ) { 
                if ( petal_length_cm < 6174969447489747968i128 ) { 
                    return 3773197899633146880i128; 
                } else { 
                    return 664082758061090560i128; 
                } 
            } else { 
               